In [60]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.nlp.tokenizers import Tokenizer
import textwrap
import os
from PIL import Image, ImageDraw, ImageFont
import pytrends
from pytrends.request import TrendReq
import json
from tqdm import tqdm

pytrend = TrendReq()

def get_google_trends_score(title):
    try:
        pytrend.build_payload([title])
        trend_data = pytrend.interest_over_time()
        point_value = trend_data[title].mean()
    except:
        point_value = 10
    
    return int(point_value)

def summarize_text(text, num_sentences):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    algorithm = LexRankSummarizer()
    summary = algorithm(parser.document, num_sentences)
    summary_text = "\n".join([str(sentence) for sentence in summary])
    return summary_text

def generate_card(title, definition, points):
    # points needs to be a string here because it is used in the draw.text function, but it is an integer in the get_google_trends_score function
    # determine the font size based on the length of the definition
    
    # definition needs to be extracted from ['definition'] to be used in the function
    if isinstance(definition, list):
        definition = definition[0]
    elif isinstance(definition, str):
        # replace '[', and ']' with ''
        definition = definition.replace('[', '')
        definition = definition.replace(']', '')
        definition = definition.replace("'", '')
        definition = definition.replace('"', '')
    
    # determine the font size based on the length of the definition (only for the body) the title font size is fixed
    font_size = 10
    title_font_size = 10
    
    # define wrapped_definition_str
    wrapped_definition_str =  textwrap.wrap(definition, width=50)

    # create the image and draw objects
    # the canvas should be 8.5 cm by 5.5 cm
    rectangle_width = 550
    rectangle_height = 850
    
    image = Image.new('RGB', (rectangle_width,rectangle_height), (255, 255, 255))
    
    # image = Image.new('RGB', (400, 300), (255, 255, 255))
    draw = ImageDraw.Draw(image)

    # select a font and draw the title in a rectangle (vertically centered)
    # add fontsize for title
    font = ImageFont.truetype('./fonts/Menlo.ttc', title_font_size, encoding='utf-8', layout_engine=ImageFont.Layout.RAQM)
    
    # find the width and height of the card
    card_width, card_height = image.size

    # draw the title in a rectangle
    draw.rectangle([(10, 10), (390, 50)])
    draw.text((20, 20), title, fill=(0, 0, 0), font=font)


    # draw the definition in a rectangle that goes from the bottom of the title rectangle to the bottom of the canvas (vertically centered) and is 50 characters wide, leaving a 10 pixel margin on the left and right and a 20 pixel margin on the bottom.
    
    # add fontsize for body
    font = ImageFont.truetype('./fonts/Menlo.ttc', font_size, encoding='utf-8', layout_engine=ImageFont.Layout.RAQM)

    # draw the definition in a rectangle
    # wrap the body text to not exceed the width of the card and the buffer space on the left and right

    # find the width and height of the card
    card_width, card_height = image.size

    # find the width and height of the title rectangle
    title_width, title_height = draw.textsize(title, font=font)

    # find the width and height of the point value rectangle (the circle)
    point_value_width, point_value_height = draw.textsize(str(points), font=font)

    # find the width and height of the definition rectangle
    definition_width, definition_height = draw.textsize(definition, font=font)

    # find the width and height of the buffer space on the left and right
    buffer_width = card_width - definition_width

    # find the width and height of the buffer space on the bottom
    buffer_height = card_height - definition_height

    # find the width and height of the whitespace margin on the left and right
    whitespace_width = buffer_width / 2

    # find the width and height of the whitespace margin on the bottom
    whitespace_height = buffer_height / 2

    # find the width and height of the whitespace margin on the top
    whitespace_height_top = whitespace_height - title_height

    # find the width and height of the whitespace margin on the left and right of the title rectangle
    whitespace_width_title = whitespace_width - title_width

    # find the width and height of the whitespace margin on the left and right of the point value rectangle
    whitespace_width_point_value = whitespace_width - point_value_width

    # find the width and height of the whitespace margin on the left and right of the definition rectangle
    whitespace_width_definition = whitespace_width - definition_width

    # find the width and height of the whitespace margin on the left and right of the point value rectangle
    whitespace_width_point_value = whitespace_width - point_value_width

    # find the width and height of the whitespace margin on the left and right of the definition rectangle
    whitespace_width_definition = whitespace_width - definition_width

    # find the width and height of the whitespace margin on the left and right of the point value rectangle

    # draw the rectangle for the definition
    draw.rectangle([(whitespace_width_title, whitespace_height_top), (card_width - whitespace_width_title, card_height - whitespace_height)])

    # draw the definition in the rectangle
    draw.text((whitespace_width_title, whitespace_height_top), definition, fill=(0, 0, 0), font=font)
    
    # draw the definition in a rectangle that goes from the bottom of the title rectangle to the bottom of the canvas (vertically centered) and is 50 characters wide, leaving a 10 pixel margin on the left and right and a 20 pixel margin on the bottom.
    # add fontsize for body
    font = ImageFont.truetype('./fonts/Menlo.ttc', font_size, encoding='utf-8', layout_engine=ImageFont.Layout.RAQM)
    
    # draw the definition in a rectangle
    # wrap the body text to not exceed the width of the card and the buffer space on the left and right
    # find the width and height of the card
    card_width, card_height = image.size

    # find the width and height of the title rectangle
    title_width, title_height = draw.textsize(title, font=font)

    # find the width and height of the point value rectangle (the circle)
    point_value_width, point_value_height = draw.textsize(str(points), font=font)

    # find the width and height of the definition rectangle
    definition_width, definition_height = draw.textsize(definition, font=font)

    # find the width and height of the buffer space on the left and right
    buffer_width = card_width - definition_width
    
    # find the width and height of the buffer space on the bottom
    buffer_height = card_height - definition_height

    # find the width and height of the whitespace margin on the left and right
    whitespace_width = buffer_width / 2

    # find the width and height of the whitespace margin on the bottom
    whitespace_height = buffer_height / 2

    # find the width and height of the whitespace margin on the top
    whitespace_height_top = whitespace_height - title_height

    # find the width and height of the whitespace margin on the left and right of the title rectangle
    whitespace_width_title = whitespace_width - title_width

    # find the width and height of the whitespace margin on the left and right of the point value rectangle
    whitespace_width_point_value = whitespace_width - point_value_width

    # find the width and height of the whitespace margin on the left and right of the definition rectangle
    whitespace_width_definition = whitespace_width - definition_width

    # find the width and height of the whitespace margin on the left and right of the point value rectangle
    whitespace_width_point_value = whitespace_width - point_value_width

    # find the width and height of the whitespace margin on the left and right of the definition rectangle
    whitespace_width_definition = whitespace_width - definition_width

    # find the width and height of the whitespace margin on the left and right of the point value rectangle

    # draw the rectangle for the definition
    draw.rectangle([(whitespace_width_title, whitespace_height_top), (card_width - whitespace_width_title, card_height - whitespace_height)])

    # draw the definition in the rectangle
    draw.text((whitespace_width_title, whitespace_height_top), definition, fill=(0, 0, 0), font=font)

    # the point value should be drawn in a circle in the bottom right corner of the card and centered within the circle (vertically and horizontally). The circle should be 1 cm in diameter and be positioned 0.5 cm from the bottom and right edges of the card. Scale the font of the point value to keep it within the circle. 
    # draw a circle around the point value
    draw.ellipse([(350, 800), (450, 900)], fill='lightblue', outline='black', width=2)
    # draw the point value in the circle
    # add fontsize for body
    # points round down to

    # start 2 cm from the top, and end 2 cm from the bottom of the card.
    # leave a 0.5 cm whitespace margin on the left and right of the card.
    draw.rectangle([(10, 60), (390, 250)])
    draw.text((20, 70), wrapped_definition_str, fill=(0, 0, 0), font=font)

    # the point value should be drawn in a circle in the bottom right corner of the card and centered within the circle (vertically and horizontally). The circle should be 1 cm in diameter and be positioned 0.5 cm from the bottom and right edges of the card. Scale the font of the point value to keep it within the circle. 
    # draw a circle around the point value
    draw.ellipse([(350, 800), (450, 900)], fill='lightblue', outline='black', width=2)
    # draw the point value in the circle
    # add fontsize for body
    # points round down to the nearest integer
    
    points = int(points)

    font = ImageFont.truetype('./fonts/Menlo.ttc', font_size, encoding='utf-8', layout_engine=ImageFont.Layout.RAQM)
    draw.text((375, 825), str(points), fill=(0, 0, 0), font=font, align='center')
    # save the image
    image.save('./card_images/{}.png'.format(len(os.listdir('./card_images/'))))

def generate_cards():
    # get cards
    with open('ppn_deck.json', 'r') as f:
        cards = json.load(f)

    # create cards directory if it doesn't exist
    if not os.path.exists('./cards'):
        os.makedirs('./cards')

    # generate cards
    for card in tqdm(cards):
        title = card['title']
        definition = card['summary_short'] if 'summary_short' in card else card['summary'] # if there is a short summary, use that, otherwise use the full summary
        definition = definition[0] if isinstance(definition, list) else summarize_text(definition, 1) # if definition is a list, take the first element, otherwise summarize the definition
        points = get_google_trends_score(title) # get points from google trends
        generate_card(title, definition, str(points))
    

generate_cards()

  0%|          | 0/954 [00:00<?, ?it/s]/opt/anaconda3/lib/python3.9/site-packages/PIL/ImageFont.py:222: UserWarning: Raqm layout was requested, but Raqm is not available. Falling back to basic layout.
  warnings.warn(
/var/folders/34/d1tlq3k91hb0lj6x90xpzb4r0000gn/T/ipykernel_81002/1822300880.py:86: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  title_width, title_height = draw.textsize(title, font=font)
/var/folders/34/d1tlq3k91hb0lj6x90xpzb4r0000gn/T/ipykernel_81002/1822300880.py:89: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  point_value_width, point_value_height = draw.textsize(str(points), font=font)
/var/folders/34/d1tlq3k91hb0lj6x90xpzb4r0000gn/T/ipykernel_81002/1822300880.py:92: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  definition_

TypeError: expected string

In [ ]:
from sumy.parsers.plaintext import PlaintextParser
# LexRankSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
# LsaSummarizer
from sumy.summarizers.lsa import LsaSummarizer
# LuhnSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
# TextRankSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
import textwrap
import os
from PIL import Image, ImageDraw, ImageFont
from tqdm import tqdm

# import minmax scaler eventually for scaling the point values between 1 and 10

import pytrends
from pytrends.request import TrendReq

pytrend = TrendReq()

def get_google_trends_score(title):
    # Get the Google Trends data for the given title
    try:
        pytrend.build_payload([title])
        trend_data = pytrend.interest_over_time()
        
        # Calculate the point value based on the average monthly search volume for the title
        point_value = trend_data[title].mean()
    except:
        point_value = 10 # If there is an error, return a point value of 10
    return point_value






def generate_card(title, definition, points):
    # determine the font size based on the length of the definition
    
    # definition needs to be extracted from ['definition'] to be used in the function
    if isinstance(definition, list):
        definition = definition[0]
    elif isinstance(definition, str):
        # replace '[', and ']' with ''
        definition = definition.replace('[', '')
        definition = definition.replace(']', '')
        definition = definition.replace("'", '')
        definition = definition.replace('"', '')
    
    body_font_size = int(len(definition) / 20)
    title_font_size = 10

    # create the image and draw objects
    # the canvas should be 8.5 cm by 5.5 cm
    rectangle_width = 550
    rectangle_height = 850
    
    image = Image.new('RGB', (rectangle_width,rectangle_height), (255, 255, 255))
    
    # image = Image.new('RGB', (400, 300), (255, 255, 255))
    draw = ImageDraw.Draw(image)

    # select a font and draw the title in a rectangle (vertically centered)
    # add fontsize for title
    font = ImageFont.truetype('./fonts/Menlo.ttc', title_font_size, encoding='utf-8', layout_engine=ImageFont.Layout.RAQM)
    
    # find the width and height of the card
    card_width, card_height = image.size

    # calculate the center of the card in terms of x and y coordinates
    card_center_x = card_width // 2
    card_center_y = card_height // 2
    
    # calculate the x and y coordinates of the top left and bottom right corners of the rectangle
    rectangle_top_left_x = card_center_x - rectangle_width // 2
    rectangle_top_left_y = card_center_y - rectangle_height // 2
    rectangle_bottom_right_x = card_center_x + rectangle_width // 2
    rectangle_bottom_right_y = card_center_y + rectangle_height // 2

    # draw the rectangle
    draw.rectangle([(rectangle_top_left_x, rectangle_top_left_y), (rectangle_bottom_right_x, rectangle_bottom_right_y)], fill='lightgrey', outline='black', width=2)
    draw.text((20, 20), title, fill=(0, 0, 0), font=font, align='center')

    # draw the definition in a rectangle that goes from the bottom of the title rectangle to the bottom of the canvas (vertically centered) and is 50 characters wide, leaving a 10 pixel margin on the left and right and a 20 pixel margin on the bottom.
    
    # add fontsize for body
    font = ImageFont.truetype('./fonts/Menlo.ttc', body_font_size, encoding='utf-8', layout_engine=ImageFont.Layout.RAQM)

    # draw the definition in a rectangle
    wrapped_definition = textwrap.wrap(definition, width=50)
    wrapped_definition_str = "\n".join(wrapped_definition)
    
    # start 2 cm from the top, and end 2 cm from the bottom of the card.
    # leave a 0.5 cm whitespace margin on the left and right of the card.
    draw.rectangle([(10, 60), (390, 250)])
    draw.text((20, 70), wrapped_definition_str, fill=(0, 0, 0), font=font)

    # the point value should be drawn in a circle in the bottom right corner of the card and centered within the circle (vertically and horizontally). The circle should be 1 cm in diameter and be positioned 0.5 cm from the bottom and right edges of the card. Scale the font of the point value to keep it within the circle. 
    # draw a circle around the point value
    draw.ellipse([(350, 800), (450, 900)], fill='lightblue', outline='black', width=2)
    # draw the point value in the circle
    # add fontsize for body
    font = ImageFont.truetype('./fonts/Menlo.ttc', body_font_size, encoding='utf-8', layout_engine=ImageFont.Layout.RAQM)
    draw.text((375, 825), str(points), fill=(0, 0, 0), font=font, align='center')
    # save the image
    image.save('./card_images/{}.png'.format(len(os.listdir('./card_images/'))))

# todo
def summarize_text(text, num_sentences):
    """
    Summarize the given text using the LSA or LexRank summarization algorithms and return the summary as a string
    """
    # create a PlaintextParser object to parse the text
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    # choose a summarization algorithm
    # algorithm = LsaSummarizer()
    algorithm = LexRankSummarizer()

    # summarize the text and return the summary as a string
    summary = algorithm(parser.document, num_sentences)
    summary_text = "\n".join([str(sentence) for sentence in summary])

    return summary_text

import textwrap


import json
# open the cards file
with open("ppn_deck_cleaned.json", "r") as read_file:
    card_deck = json.load(read_file)

# create a new card deck (cards with point values from Google Trends)
new_card_deck = []

# loop through the cards in the card deck
for card in tqdm(card_deck):
    # get the title and definition
    title = card["title"]
    definition = summarize_text(card["summary"],2)
    # check if the definition has more than 2 sentences
    if len(definition.split(".")) > 2:
        # if so, use the first 2 sentences
        definition = ".".join(definition.split(".")[:2])
    
    # get the Google Trends score for the title
    points = get_google_trends_score(str(title))
    # round the points to the nearest integer
    points = int(round(points))
    # create a new card with the title, definition, and point value
    new_card = {
        "title": title,
        "definition": definition,
        "points": points
    }
    # add the new card to the new card deck
    card_generated = generate_card(title, definition, str(points)) # generate card image
    #print(f'title: {title}, definition: {definition}, points: {points}')
    #create_card(title, definition, str(points), 400, 150) # generate card image




  0%|          | 0/931 [00:00<?, ?it/s]/var/folders/34/d1tlq3k91hb0lj6x90xpzb4r0000gn/T/ipykernel_99512/3586156148.py:69: DeprecationWarning: LAYOUT_RAQM is deprecated and will be removed in Pillow 10 (2023-07-01). Use Layout.RAQM instead.
  font = ImageFont.truetype('./fonts/NewYork.ttf', title_font_size, encoding='utf-8', layout_engine=ImageFont.LAYOUT_RAQM)
/opt/anaconda3/lib/python3.9/site-packages/PIL/ImageFont.py:222: UserWarning: Raqm layout was requested, but Raqm is not available. Falling back to basic layout.
  warnings.warn(
/var/folders/34/d1tlq3k91hb0lj6x90xpzb4r0000gn/T/ipykernel_99512/3586156148.py:91: DeprecationWarning: LAYOUT_RAQM is deprecated and will be removed in Pillow 10 (2023-07-01). Use Layout.RAQM instead.
  font = ImageFont.truetype('./fonts/NewYork.ttf', body_font_size, encoding='utf-8', layout_engine=ImageFont.LAYOUT_RAQM)
/var/folders/34/d1tlq3k91hb0lj6x90xpzb4r0000gn/T/ipykernel_99512/3586156148.py:107: DeprecationWarning: LAYOUT_RAQM is deprecated and w

In [ ]:
from nltk.metrics.distance import edit_distance
import json
import nltk
from nltk.corpus import wordnet
import random
import nltk
from tqdm import tqdm
from nltk.corpus import wordnet
stopwords = nltk.corpus.stopwords.words('english')
def find_synonyms(word):
    synonyms = []
    lemmatizer = nltk.WordNetLemmatizer()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonym = lemma.name()
            synonyms.append(synonym)
    return set(synonyms)

def replace_similar_words(title, definition, threshold=3):
    similar_words = find_similar_words(title, definition, threshold)
    
    # replace each similar word with a random synonym
    for word in similar_words:
        synonyms = find_synonyms(word)
        synonym = random.choice(list(synonyms))  # choose a random synonym from the set
        print(f"Replacing {word} with {synonym}")
        definition = definition.replace(word, synonym)
        
    return definition


# Example usage
print(find_synonyms("abolitionism"))
# Output: {'abolition', 'abolitionary', 'abolitionism', 'manumission', 'emancipation'}

# open the card deck file
with open("ppn_deck.json", "r") as read_file:
    card_deck = json.load(read_file)
def find_similar_words(title, definition, threshold=3):
    # split the definition into a list of words
    if isinstance(definition, list):
        words = definition
    else:
        words = definition.split()
    
    # use a list comprehension to create a list of words with a Levenshtein distance less than the threshold
    similar_words = [word for word in words if edit_distance(title, word) <= threshold]
    
    return similar_words

def replace_similar_words(title, definition, threshold=3):
    similar_words = find_similar_words(title, definition, threshold)
    if isinstance(definition, list):
        definition = " ".join(definition)
    # replace each similar word with a synonym
    for word in similar_words:
        if len(word) < 3 or word in stopwords:
            continue
        try:
            synonyms = find_synonyms(word) # find a synonym for the word -> set
            # extract the synonym from the set
            synonyms = list(synonyms)
            synonym = synonyms[0]
            #print(f"Replacing {word} with {synonym}")
            definition = definition.replace(word, str(synonym))
        except Exception as e:
            #print(e)
            #print("No synonyms found for word: ", word)
            pass
    return definition

# iterate through each card and replace the similar words in the summary_short with synonyms
for card in tqdm(card_deck):
    title = card['title']
    definition = card['summary']
    definition = replace_similar_words(title, definition)
    
    # update the definition in the card dictionary
    card['summary_short'] = definition


{'abolitionism'}


100%|██████████| 931/931 [00:12<00:00, 73.19it/s] 


In [74]:
import textwrap
import os
from PIL import Image, ImageDraw, ImageFont
with open("ppn_deck_cleaned.json", "w") as write_file:
    json.dump(card_deck, write_file, indent=4)

def get_google_trends_score(title):
    try:
        pytrend.build_payload([title])
        trend_data = pytrend.interest_over_time()
        point_value = trend_data[title].mean()
    except:
        point_value = 10
    
    return int(point_value)

def generate_card(title, definition, points):
    # determine the font size based on the length of the definition
    # font_size = int(len(definition) / 20)
    font_size = max(30, int(len(definition) / 20))
    # create the image and draw objects
    # set the canvas size to 8.5 cm by 5.5 cm
    image = Image.new('RGB', (550, 850), (255, 255, 255))
    draw = ImageDraw.Draw(image)

    # select a font and draw the title in a rectangle
    font = ImageFont.truetype('./fonts/Menlo.ttc', 15)
    draw.rectangle([(10, 10), (540, 50)], fill='lightgrey')
    draw.text((20, 20), title, fill=(0, 0, 0), font=font)

    # draw the definition in a rectangle, and soft wrap the text. Don't exceed 40 characters per line. 
    # wrapped_definition = textwrap.wrap(definition, width=40)
    definition = str(definition) if isinstance(definition, str) else definition[0]
    wrapped_definition = textwrap.fill(definition, width=80)
    wrapped_definition_str = "\n".join(wrapped_definition)
    font = ImageFont.truetype('./fonts/Menlo.ttc', font_size)
    draw.text((20, 70), wrapped_definition, fill=(0, 0, 0))

    # draw a circle around the point value
    draw.ellipse([(520, 820), (540, 840)], fill='lightblue')
    # draw the point value

    font = ImageFont.truetype('./fonts/Menlo.ttc', font_size)
    draw.text((525, 825), str(points), fill=(0, 0, 0))

    # save the image
    image.save('./card_images/{}.png'.format(len(os.listdir('./card_images/'))))

#^ Example usage
card = random.choice(card_deck)
print(card)
summary = card['summary'][1] if isinstance(card['summary'], list) else card['summary']
# summarize the definition with the summarize function
summary = summarize_text(summary, 2) if isinstance(summary, str) else summary # if the summary is a list, then it's already been summarized
generate_card(str(card['title']), summary, points=get_google_trends_score(card['title']))
# generate_card('test title', 'test definition', 10)

{'title': 'Thank God for Mississippi', 'summary': ['Thank God for Mississippi', 'Thank God for Mississippi is an adage used in the United States, particularly in the South, that is generally used when discussing rankings of U.S. states. Since the U.S. state of Mississippi commonly ranks at or near the bottom of such rankings, residents of other states also ranking near the bottom may say, Thank God for Mississippi, since the presence of that state in 50th place spares them the shame of being ranked last.Examples include rankings of educational achievement, business opportunities, obesity rates, overall health, the poverty rate, life expectancy, or other objective or subjective criteria of the quality of life or government in the 50 states. The phrase is in use even among state government officials and journalists, though occasionally with a slight twist.Mississippis poor reputation is such a common trope in American culture that when Mississippi does indeed rank well in something, the 

In [75]:
!pip install turtle

  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [7 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/private/var/folders/34/d1tlq3k91hb0lj6x90xpzb4r0000gn/T/pip-install-txzsu4v8/turtle_41413f17c2b8408cbad6a753ce82f0f8/setup.py", line 40
          except ValueError, ve:
                           ^
      SyntaxError: invalid syntax
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [79]:
!pip install weasyprint
# !sudo apt-get install libcairo2 pkg-config python3-dev


Password:
sudo: a password is required


In [78]:
import os
from weasyprint import HTML

def generate_card(title, body, points):
    # create the HTML for the game card
    html = f'''
        <html>
            <head>
                <style>
                    .card {{
                        width: 85mm;
                        height: 55mm;
                        background-color: white;
                        border: 1px solid black;
                        display: flex;
                        flex-direction: column;
                    }}
                    .title {{
                        background-color: lightgrey;
                        font-size: 15pt;
                        padding: 5mm;
                    }}
                    .body {{
                        font-size: 12pt;
                        padding: 5mm;
                        flex-grow: 1;
                    }}
                    .points {{
                        background-color: lightblue;
                        font-size: 30pt;
                        display: flex;
                        align-items: center;
                        justify-content: center;
                    }}
                </style>
            </head>
            <body>
                <div class="card">
                    <div class="title">{title}</div>
                    <div class="body">{body}</div>
                    <div class="points">{points}</div>
                </div>
            </body>
        </html>
    '''

    # write the HTML to a file
    with open('game_card.html', 'w') as f:
        f.write(html)

    # convert the HTML file to a PNG image
    HTML(string=html).write_png('game_card.png')

# example usage
generate_card('Title of the Card', 'Body text of the card', 10)



-----

WeasyPrint could not import some external libraries. Please carefully follow the installation steps before reporting an issue:
https://doc.courtbouillon.org/weasyprint/stable/first_steps.html#installation
https://doc.courtbouillon.org/weasyprint/stable/first_steps.html#troubleshooting 

-----



OSError: cannot load library 'pango-1.0-0': dlopen(pango-1.0-0, 0x0002): tried: 'pango-1.0-0' (no such file), '/System/Volumes/Preboot/Cryptexes/OSpango-1.0-0' (no such file), '/opt/anaconda3/lib/pango-1.0-0' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/anaconda3/lib/pango-1.0-0' (no such file), '/opt/anaconda3/lib/pango-1.0-0' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/anaconda3/lib/pango-1.0-0' (no such file), '/opt/anaconda3/lib/python3.9/site-packages/../../pango-1.0-0' (no such file), '/opt/anaconda3/bin/../lib/pango-1.0-0' (no such file), '/usr/lib/pango-1.0-0' (no such file, not in dyld cache), 'pango-1.0-0' (no such file), '/usr/local/lib/pango-1.0-0' (no such file), '/usr/lib/pango-1.0-0' (no such file, not in dyld cache).  Additionally, ctypes.util.find_library() did not manage to locate a library called 'pango-1.0-0'